# 01. 어텐션 메커니즘 (Attention Mechanism) 심층 분석

## 💡 "어텐션(Attention)"이 뭔가요?
사람이 무언가를 볼 때 모든 것에 똑같이 집중하지 않죠? 중요한 부분에 **'집중(Attention)'**하고 나머지는 흐릿하게 인식합니다. 딥러닝 모델에게도 이런 능력을 심어준 게 바로 어텐션 메커니즘입니다.

### 📚 핵심 비유: 도서관의 사서
어텐션을 **'도서관에서 책 찾기'**에 비유해볼게요.

*   **Query (Q, 질문)**: _"금융 위기에 대한 책 찾아줘"_ (내가 찾고자 하는 의도)
*   **Key (K, 분류표/꼬리표)**: _[경제, 역사, 소설, 과학...]_ (정보를 찾기 위한 인덱스)
*   **Value (V, 책 내용)**: _[경제학 원론 내용, 조선왕조실록 내용...]_ (실제 담고 있는 정보)

**작동 원리:**
1.  내 질문(Q)과 분류표(K)를 비교해서 얼마나 연관성 있는지 **유사도 점수(Attention Score)**를 매깁니다.
2.  점수가 높은 책일수록 더 많이 참고하라고 **가중치(Weight)**를 줍니다.
3.  이 가중치대로 책 내용(V)을 섞어서 **최종 요약본(Context Vector)**을 만듭니다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# 시각화를 위한 설정 (선택 사항)
import matplotlib.pyplot as plt
import seaborn as sns

## 1. 입력 데이터 준비
3개의 단어로 이루어진 문장이 하나 있다고 가정해봅시다. (Batch=1, Seq_len=3)
각 단어는 4차원 벡터로 표현됩니다. (Embedding_dim=4)

In [2]:
# 입력 x: (batch_size, seq_len, embedding_dim)
# 예: ["I", "love", "AI"] 라는 문장이 임베딩된 상태
x = torch.tensor([
    [
        [1.0, 0.0, 1.0, 0.0],   # 단어 1
        [0.0, 2.0, 0.0, 2.0],   # 단어 2
        [1.0, 1.0, 1.0, 1.0]    # 단어 3
    ]
]) 

print('입력 x 형태:', x.shape)  # torch.Size([1, 3, 4])

## 2. Q, K, V 생성 (Linear Transformation)
입력 벡터 $x$를 그대로 쓰는 게 아니라, 각각의 역할에 맞게 변신시켜줍니다.
마치 재료($x$)를 가지고 요리사($W$)들이 각자 다른 요리(Q, K, V)를 준비하는 것과 같아요.

*   **$W^Q$**: 질문(Query)을 만드는 가중치
*   **$W^K$**: 꼬리표(Key)를 만드는 가중치
*   **$W^V$**: 내용물(Value)을 다듬는 가중치

In [3]:
d_model = 4  # 임베딩 차원

# 선형 변환 레이어 정의 (bias는 편의상 끔)
W_q = nn.Linear(d_model, d_model, bias=False)
W_k = nn.Linear(d_model, d_model, bias=False)
W_v = nn.Linear(d_model, d_model, bias=False)

# 입력 x를 통과시켜 Q, K, V 생성
Q = W_q(x)  # (1, 3, 4)
K = W_k(x)  # (1, 3, 4)
V = W_v(x)  # (1, 3, 4)

print("Query shape :", Q.shape)
print("Key shape   :", K.shape)
print("Value shape :", V.shape)

## 3. Scaled Dot-Product Attention
어텐션의 핵심 공식입니다!

$$ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V $$

### 단계별로 뜯어보기

#### 3-1. 점수 매기기 (QK^T)
Query와 Key를 **내적(Dot Product)**합니다. 내적은 두 벡터가 얼마나 비슷한지를 나타내는 지표죠.
- 내적 값이 크다? -> "어? 내 질문이랑 딱 맞는 책이네!" (관련성 높음)
- 내적 값이 작다? -> "별로 상관없는 책이군." (관련성 낮음)

In [4]:
# Q(1, 3, 4)와 K의 전치행렬(1, 4, 3)을 곱함 -> (1, 3, 3)
# 결과 행렬의 의미: [단어1과 단어1의 관계, 단어1과 단어2의 관계, ...]
attn_scores = torch.matmul(Q, K.transpose(-2, -1)) 

print("어텐션 스코어 형태:", attn_scores.shape)

#### 3-2. 스케일링 (Scaling)
벡터 차원($d_k$)이 커지면 내적 값도 덩달아 엄청 커질 수 있어요.
값이 너무 커지면 Softmax를 거칠 때 기울기(Gradient)가 사라지는 문제가 생깁니다.
그래서 차원의 제곱근($\sqrt{d_k}$)으로 나누어 값을 좀 진정시켜줍니다.

In [5]:
d_k = Q.size(-1)
scaled_scores = attn_scores / math.sqrt(d_k)

print("스케일링 된 스코어 (일부):\n", scaled_scores[0].detach().numpy())

#### 3-3. 확률 분포 변환 (Softmax)
점수들을 확률(%)로 바꿉니다. 모든 점수의 합이 1이 되도록 만들어서, "이 단어는 70% 참고하고, 저 단어는 30% 참고해라"라는 명확한 비율을 만듭니다.

In [6]:
attn_weights = F.softmax(scaled_scores, dim=-1)

print("어텐션 가중치 (확률 분포, 행별 합=1):\n", attn_weights[0].detach().numpy())

#### 3-4. 최종 결과 생성 (Value 가중합)
구해둔 확률($\text{Attention Weights}$)을 실제 정보($V$)에 곱해서 더합니다.
중요한 정보는 진하게 남고, 중요하지 않은 정보는 희미해져서 섞이게 되죠.

이게 바로 **Self-Attention**의 마법입니다! 문장 내의 단어들이 서로 문맥을 파악해서 정보를 섞는 과정이죠.

In [7]:
output = torch.matmul(attn_weights, V)

print("최종 출력 형태 :", output.shape)
print("최종 출력 값 (첫번째 단어에 대한 요약):\n", output[0, 0].detach().numpy())

---
## 4. 멀티 헤드 어텐션 (Multi-Head Attention)

혼자서 공부하는 것보다, 여러 명의 전문가가 각자 다른 관점에서 분석하면 더 좋겠죠?
*   **헤드 1**: "나는 주어와 동사의 관계를 집중해서 볼게."
*   **헤드 2**: "나는 형용사가 뭘 꾸며주는지 볼게."
*   **헤드 3**: "나는 시제 정보를 볼게."

이렇게 입력을 여러 개(`num_heads`)로 쪼개서 병렬로 어텐션을 수행하는 것이 **Multi-Head Attention**입니다.

In [8]:
# 입력 차원을 조금 키워봅시다 (8차원)
x_large = torch.randn(1, 3, 8)  # 배치 1, 길이 3, 차원 8
B, T, d_model = x_large.shape

# 헤드 설정
n_head = 4
d_head = d_model // n_head  # 각 헤드가 담당할 차원 크기 (8 / 4 = 2)

print(f"전체 차원: {d_model}, 헤드 개수: {n_head}, 헤드 당 차원: {d_head}")

### 4-1. Q, K, V 헤드 나누기 (Splitting Heads)
행렬 차원을 조작해서 여러 개의 헤드로 쪼갭니다. `view`와 `transpose`를 사용해요.

In [9]:
# Q, K, V 생성을 위한 선형층
W_q = nn.Linear(d_model, d_model, bias=False)
W_k = nn.Linear(d_model, d_model, bias=False)
W_v = nn.Linear(d_model, d_model, bias=False)

# 1. 선형 변환
q = W_q(x_large)  # (1, 3, 8)
k = W_k(x_large)
v = W_v(x_large)

# 2. 헤드 나누기 (Split Heads)
# (Batch, Time, D_model) -> (Batch, Time, Head, D_head) -> (Batch, Head, Time, D_head)
# transpose(1, 2)를 해주는 이유: 
# 어텐션 계산(matmul)은 뒤쪽 2개 차원(Time, D_head)끼리 이루어져야 하므로 Head 차원을 앞으로 빼줍니다.
q = q.view(B, T, n_head, d_head).transpose(1, 2)
k = k.view(B, T, n_head, d_head).transpose(1, 2)
v = v.view(B, T, n_head, d_head).transpose(1, 2)

print("헤드 분할 후 Q shape:", q.shape)  # (1, 4, 3, 2)

### 4-2. 헤드별 어텐션 수행
이제 4개의 헤드가 각자 알아서 자기 몫의 어텐션을 계산합니다. (병렬 처리)

In [10]:
# (B, Head, T, D) * (B, Head, D, T) -> (B, Head, T, T)
scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_head)
weights = F.softmax(scores, dim=-1)

# 가중합
# (B, Head, T, T) * (B, Head, T, D) -> (B, Head, T, D)
attn_val = torch.matmul(weights, v)

print("헤드별 어텐션 결과:", attn_val.shape)

### 4-3. 다시 하나로 합치기 (Concat)
4명의 전문가가 내놓은 결과를 다시 하나로 이어 붙입니다. 원래 차원(8차원)으로 복구되는 거죠.

In [11]:
# (Batch, Head, Time, D_head) -> (Batch, Time, Head, D_head)
attn_val = attn_val.transpose(1, 2).contiguous()

# (Batch, Time, D_model)로 쫙 펴주기 (Flatten)
output = attn_val.view(B, T, d_model)

# 마지막으로 한 번 더 잘 섞어주는 선형층 (W_o) 통과
W_o = nn.Linear(d_model, d_model, bias=False)
final_output = W_o(output)

print("최종 출력 형태 (원상복구):", final_output.shape)

### ✨ 요약
1.  **Q, K, V**: 질문, 꼬리표, 내용물로 변환.
2.  **Scaled Dot-Product**: 유사도 점수 계산 및 확률 변환.
3.  **Multi-Head**: 여러 관점에서 동시에 분석하기 위해 쪼개서 계산 후 병합.

이 메커니즘이 바로 **Transformer**가 문맥을 기가 막히게 파악하는 비결입니다! 👍